## 🌐 **Google Drive Connection**

In [ ]:
# 1️⃣ Import delle librerie principali
import matplotlib.pyplot as plt
import pandas as pd
import logging
import random
import numpy as np
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import copy
import shutil
from itertools import product
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import os

# Definisci la directory principale nel workspace di Colab
current_dir = '/content/Arialeto2'

# Crea la directory se non esiste
os.makedirs(current_dir, exist_ok=True)

# Spostati nella directory del progetto
os.chdir(current_dir)
print(f"Working directory set to: {os.getcwd()}")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/.shortcut-targets-by-id/1-GldjmW2UkrRzoKRsp0d_0BiDCphSh44/Arialeto2


##PATCHES

In [ ]:
from pathlib import Path
from PIL import Image


img_dir = Path("clean_train_data")
mask_dir = Path("clean_train_masks")

img_paths = sorted(img_dir.glob("img_*.png"))
mask_paths = sorted(mask_dir.glob("mask_*.png"))

assert len(img_paths) == len(mask_paths)
for i_img, i_mask in zip(img_paths, mask_paths):
    assert i_img.stem.replace("img_", "") == i_mask.stem.replace("mask_", "")

# modello
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
def save_patches_to_disk(img_paths, mask_paths, output_dir,
                        patch_size=244, stride=122, min_mask_ratio=0.01,
                        max_patches_per_slide=500):
    """
    Salva tutte le patch come IMMAGINI in cartella sul Drive
    Nomi: img_0000_patch_007.png (NO label)
    """

    # Crea cartella output
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    total_patches_saved = 0

    for i, (img_path, mask_path) in enumerate(zip(img_paths, mask_paths)):
        slide_name = os.path.basename(img_path)  # 'img_0000.png'
        slide_stem = os.path.splitext(slide_name)[0]  # 'img_0000'

        # Carica immagine e mask
        img_array = np.array(Image.open(img_path).convert('RGB'))
        mask_array = np.array(Image.open(mask_path)) > 0

        h, w = img_array.shape[:2]
        valid_positions = []

        # Trova posizioni valide
        for y in range(0, max(0, h - patch_size + 1), stride):
            for x in range(0, max(0, w - patch_size + 1), stride):
                patch_mask = mask_array[y:y+patch_size, x:x+patch_size]
                if patch_mask.mean() > min_mask_ratio:
                    valid_positions.append((y, x))

        n_valid = len(valid_positions)
        n_take = min(n_valid, max_patches_per_slide)

        print(f"{slide_name}: {n_valid}→{n_take} patch")

        if n_take > 0:
            # Campiona
            sample_indices = np.random.choice(n_valid, n_take, replace=False)

            for local_id, pos_idx in enumerate(sample_indices):
                y, x = valid_positions[pos_idx]

                # Estrai patch
                patch_array = img_array[y:y+patch_size, x:x+patch_size]
                patch_img = Image.fromarray(patch_array.astype(np.uint8))

                # Nome file SEMPLICE: img_0000_patch_007.png
                patch_filename = f"{slide_stem}_patch_{local_id:03d}.png"
                patch_path = Path(output_dir) / patch_filename

                # Salva
                patch_img.save(patch_path)
                total_patches_saved += 1

        if i % 50 == 0:
            print(f"Progresso: {i+1}/{len(img_paths)} slide, {total_patches_saved} patch")

    print(f"\nCOMPLETATO!")
    print(f"Cartella: {output_dir}")
    print(f"Patch salvate: {total_patches_saved}")

    patch_files = list(Path(output_dir).glob("*.png"))
    print(f"Verifica: {len(patch_files)} file trovati")

    return total_patches_saved

patch_size = 224
stride = 112
min_mask_ratio=0.01
max_patches_per_slide=1000

# USO
output_drive_dir = f"clean_patches"
total_saved = save_patches_to_disk(
    img_paths=[str(p) for p in img_paths],
    mask_paths=[str(p) for p in mask_paths],
    output_dir=output_drive_dir,
    patch_size=patch_size,
    stride=stride,
    min_mask_ratio=min_mask_ratio,
    max_patches_per_slide=max_patches_per_slide
)

img_0000.png: 6→6 patch
Progresso: 1/581 slide, 6 patch
img_0002.png: 20→20 patch
img_0003.png: 11→11 patch
img_0004.png: 9→9 patch
img_0006.png: 20→20 patch
img_0007.png: 15→15 patch
img_0009.png: 35→35 patch
img_0010.png: 12→12 patch
img_0011.png: 16→16 patch
img_0013.png: 24→24 patch
img_0014.png: 19→19 patch
img_0015.png: 26→26 patch
img_0016.png: 36→36 patch
img_0017.png: 28→28 patch
img_0019.png: 16→16 patch
img_0021.png: 12→12 patch
img_0023.png: 26→26 patch
img_0024.png: 38→38 patch
img_0025.png: 8→8 patch
img_0026.png: 27→27 patch
img_0029.png: 18→18 patch
img_0030.png: 13→13 patch
img_0031.png: 58→58 patch
img_0032.png: 19→19 patch
img_0033.png: 17→17 patch
img_0034.png: 21→21 patch
img_0035.png: 13→13 patch
img_0037.png: 17→17 patch
img_0038.png: 14→14 patch
img_0039.png: 24→24 patch
img_0040.png: 31→31 patch
img_0041.png: 12→12 patch
img_0042.png: 32→32 patch
img_0043.png: 20→20 patch
img_0045.png: 12→12 patch
img_0046.png: 11→11 patch
img_0049.png: 12→12 patch
img_0050.png

## TEST PATCHES

In [ ]:
img_dir = Path("test_data")
mask_dir = Path("test_data")

img_paths = sorted(img_dir.glob("img_*.png"))
mask_paths = sorted(mask_dir.glob("mask_*.png"))

assert len(img_paths) == len(mask_paths)
for i_img, i_mask in zip(img_paths, mask_paths):
    assert i_img.stem.replace("img_", "") == i_mask.stem.replace("mask_", "")

# modello
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
def save_patches_to_disk(img_paths, mask_paths, output_dir,
                        patch_size=244, stride=122, min_mask_ratio=0.01,
                        max_patches_per_slide=500):
    """
    Salva tutte le patch come IMMAGINI in cartella sul Drive
    Nomi: img_0000_patch_007.png
    """

    # Crea cartella output
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    total_patches_saved = 0

    for i, (img_path, mask_path) in enumerate(zip(img_paths, mask_paths)):
        slide_name = os.path.basename(img_path)  # 'img_0000.png'
        slide_stem = os.path.splitext(slide_name)[0]  # 'img_0000'

        # Carica immagine e mask
        img_array = np.array(Image.open(img_path).convert('RGB'))
        mask_array = np.array(Image.open(mask_path)) > 0

        h, w = img_array.shape[:2]
        valid_positions = []

        # Trova posizioni valide
        for y in range(0, max(0, h - patch_size + 1), stride):
            for x in range(0, max(0, w - patch_size + 1), stride):
                patch_mask = mask_array[y:y+patch_size, x:x+patch_size]
                if patch_mask.mean() > min_mask_ratio:
                    valid_positions.append((y, x))

        n_valid = len(valid_positions)
        n_take = min(n_valid, max_patches_per_slide)

        print(f"{slide_name}: {n_valid}→{n_take} patch")

        if n_take > 0:
            # Campiona
            sample_indices = np.random.choice(n_valid, n_take, replace=False)

            for local_id, pos_idx in enumerate(sample_indices):
                y, x = valid_positions[pos_idx]

                # Estrai patch
                patch_array = img_array[y:y+patch_size, x:x+patch_size]
                patch_img = Image.fromarray(patch_array.astype(np.uint8))

                # Nome file SEMPLICE: img_0000_patch_007.png
                patch_filename = f"{slide_stem}_patch_{local_id:03d}.png"
                patch_path = Path(output_dir) / patch_filename

                # Salva
                patch_img.save(patch_path)
                total_patches_saved += 1

        if i % 50 == 0:
            print(f"Progresso: {i+1}/{len(img_paths)} slide, {total_patches_saved} patch")

    print(f"\nCOMPLETATO!")
    print(f"Cartella: {output_dir}")
    print(f"Patch salvate: {total_patches_saved}")

    patch_files = list(Path(output_dir).glob("*.png"))
    print(f"Verifica: {len(patch_files)} file trovati")

    return total_patches_saved

patch_size = 224
stride = 112
min_mask_ratio=0
max_patches_per_slide=1000

# USO
output_drive_dir = f"test_patches"
total_saved = save_patches_to_disk(
    img_paths=[str(p) for p in img_paths],
    mask_paths=[str(p) for p in mask_paths],
    output_dir=output_drive_dir,
    patch_size=patch_size,
    stride=stride,
    min_mask_ratio=min_mask_ratio,
    max_patches_per_slide=max_patches_per_slide
)


img_0000.png: 19→19 patch
Progresso: 1/477 slide, 19 patch
img_0001.png: 23→23 patch
img_0002.png: 54→54 patch
img_0003.png: 38→38 patch
img_0004.png: 27→27 patch


KeyboardInterrupt: 